# Sephora Product Analysis - Part 1 Data Cleaning
---

## Overview 
- Imported the necessary libraries
- Loaded the reviews’ data and merged it with the product information to create the final dataframe
- Implemented a function to perform basic cleaning on the dataframe
- Verified consistency between categorical values and numeric values
- Saved the cleaned dataframe into a CSV file for further use


### Imports

In [41]:
# Imports needed for cleaning
import pandas as pd
import numpy as np

### Load in the data

In [42]:
# Load in review's datasets and product information
review_1 = pd.read_csv('../data/reviews_0_250.csv', low_memory=False)
review_2 = pd.read_csv('../data/reviews_250_500.csv', low_memory= False)
review_3 = pd.read_csv('../data/reviews_500_750.csv', low_memory=False)
review_4 = pd.read_csv('../data/reviews_750_1000.csv', low_memory= False)
review_5 = pd.read_csv('../data/reviews_1000_1500.csv', low_memory=False)
review_6 = pd.read_csv('../data/reviews_1500_end.csv', low_memory= False)
product_info = pd.read_csv('../data/product_info.csv')

In [43]:
# Merge all review datasest into single dataframe
review_df = pd.concat([review_1, review_2, review_3, review_4, review_5, review_6])

In [44]:
# Add product info by merge the datasets
df = pd.merge(review_df, product_info, on="product_id")
df.head()

,Unnamed: 0,author_id,rating_x,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,...,1,0,0,['Clean at Sephora'],Skincare,Cleansers,NaN,0,NaN,NaN
1,1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,...,0,0,1,"['allure 2019 Best of Beauty Award Winner', 'C...",Skincare,Lip Balms & Treatments,NaN,3,24.0,24.0
2,2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,...,0,0,1,"['allure 2019 Best of Beauty Award Winner', 'C...",Skincare,Lip Balms & Treatments,NaN,3,24.0,24.0
3,3,6083038851,5,1.0,NaN,0,0,0,2023-03-20,I’ve always loved this formula for a long time...,...,0,0,1,"['allure 2019 Best of Beauty Award Winner', 'C...",Skincare,Lip Balms & Treatments,NaN,3,24.0,24.0
4,4,47056667835,5,1.0,NaN,0,0,0,2023-03-20,"If you have dry cracked lips, this is a must h...",...,0,0,1,"['allure 2019 Best of Beauty Award Winner', 'C...",Skincare,Lip Balms & Treatments,NaN,3,24.0,24.0


### Clean Data

In [45]:
# Shape before cleaning
df.shape

(1301136, 45)

In [46]:
# Inspect data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1301136 entries, 0 to 1301135
Data columns (total 45 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Unnamed: 0                1301136 non-null  int64  
 1   author_id                 1301136 non-null  object 
 2   rating_x                  1301136 non-null  int64  
 3   is_recommended            1107162 non-null  float64
 4   helpfulness               631670 non-null   float64
 5   total_feedback_count      1301136 non-null  int64  
 6   total_neg_feedback_count  1301136 non-null  int64  
 7   total_pos_feedback_count  1301136 non-null  int64  
 8   submission_time           1301136 non-null  object 
 9   review_text               1299520 non-null  object 
 10  review_title              930754 non-null   object 
 11  skin_tone                 1103798 non-null  object 
 12  eye_color                 1057734 non-null  object 
 13  skin_type                 1

In [47]:
# Create a function that performs basic cleaning on the dataframe
def clean_df(df):
    # Remove unnecessary columns for our analysis
    df = df.drop(columns = ['Unnamed: 0','child_min_price','child_max_price','child_count', 'out_of_stock','online_only','limited_edition',
                       'sephora_exclusive','sale_price_usd','value_price_usd','variation_desc','tertiary_category', 'new','highlights', 'brand_id', 'primary_category'])
    # Drop duplicates, if any
    df = df.drop_duplicates()
    
    # Drop columns that have been duplicated from the merge
    df = df.drop(columns = ['rating_y','brand_name_y','product_name_y','price_usd_y'])

    # Drop columns with more than 50% nulls
    df = df.dropna(axis='columns', thresh=len(df)/2)
    
    # Remove rows that have 50% missng information (9 missing values per row)
    df.dropna(thresh=df.shape[1] - 9, inplace =True)
    
    # If row contain no information in text column , drop row
    df.dropna(subset=['review_text'], inplace = True)

    # Impute nulls in 'is_recommended' column based on 'rating'
    df.loc[df['rating_x'] >= 4, 'is_recommended'] = df.loc[df['rating_x'] >= 4, 'is_recommended'].fillna(1)
    df.loc[df['rating_x'] <= 3, 'is_recommended'] = df.loc[df['rating_x'] <= 3, 'is_recommended'].fillna(0)

    # Impute nulls in 'review_title' with 'missing'
    df['review_title'] = df['review_title'].fillna('missing')

    # Impute remaining nulls with 'Not specified'
    df = df.fillna('Not specified')
    
    # Convert columns into correct datatypes
    df['is_recommended'] = df['is_recommended'].astype(int)
    df['price_usd_x'] = df['price_usd_x'].astype(int)
    df['submission_time'] = pd.to_datetime(df['submission_time'])
    # Rename col and change dtype
    df['reviews'] = df['reviews'].astype(int)
    
    # Rename a few cols names
    df.rename(columns={'reviews': 'reviews_count',
                  'product_name_x': 'product_name',
                  'brand_name_x': 'brand_name',
                  'price_usd_x': 'price_usd',
                  'rating_x': 'rating',
                  'secondary_category':'category'}, inplace=True)
    
    if df.isna().sum().sum() == 0:
        print('DataFrame is cleaned !')
    
    return df

In [48]:
# Apply the function
df = clean_df(df)

DataFrame is cleaned !


In [49]:
# Check shape after cleaning
df.shape

(1092737, 24)

#### Verify Consistency between Categorical and Numeric values

In [50]:
# Check for inconsistencies in categorical values
obj_cols = df.dtypes[df.dtypes=='object'].index

# Loop to examine the cat columns
for col in obj_cols:
  print(f'Column: {col}')
  print(df[col].value_counts())
  print('\n')

Column: author_id
1696370280     292
1288462295     182
7375781965     155
1930716686     155
2330399812     151
              ... 
24423289627      1
31850342814      1
1327966790       1
2049548170       1
11519076605      1
Name: author_id, Length: 502561, dtype: int64


Column: review_text
I love ALL of the Dr. Jart masks so much!!! They are all super hydrating & make the face look breathtakingly beautiful!!!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [51]:
# Replace values convey the same info
df['skin_tone']= df['skin_tone'].replace('notSureST', 'Not specified')
df['eye_color']= df['eye_color'].replace('Grey', 'gray')

In [52]:
# Find rows that contain letters in author_id and replace with 0's
# added regex=True to get rid of warning
df['author_id'] = df['author_id'].str.replace('[a-zA-Z]', '0', regex=True)

In [53]:
# Check for outliers in numeric columns
round(df.describe(),2)

,rating,is_recommended,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,price_usd,loves_count,reviews_count
count,1092737.00,1092737.00,1092737.00,1092737.00,1092737.00,1092737.00,1092737.00,1092737.00
mean,4.30,0.84,4.18,0.89,3.29,48.97,106744.30,2159.61
std,1.15,0.37,22.73,5.29,19.69,40.09,167544.72,2524.89
min,1.00,0.00,0.00,0.00,0.00,3.00,0.00,1.00
25%,4.00,1.00,0.00,0.00,0.00,25.00,16632.00,552.00
50%,5.00,1.00,0.00,0.00,0.00,39.00,48739.00,1350.00
75%,5.00,1.00,3.00,1.00,3.00,62.00,123439.00,2911.00
max,5.00,1.00,5464.00,1159.00,5050.00,1900.00,1081315.00,16118.00


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1092737 entries, 0 to 1301135
Data columns (total 24 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   author_id                 1092737 non-null  object        
 1   rating                    1092737 non-null  int64         
 2   is_recommended            1092737 non-null  int64         
 3   total_feedback_count      1092737 non-null  int64         
 4   total_neg_feedback_count  1092737 non-null  int64         
 5   total_pos_feedback_count  1092737 non-null  int64         
 6   submission_time           1092737 non-null  datetime64[ns]
 7   review_text               1092737 non-null  object        
 8   review_title              1092737 non-null  object        
 9   skin_tone                 1092737 non-null  object        
 10  eye_color                 1092737 non-null  object        
 11  skin_type                 1092737 non-null  object

### Save Final Clean Dataframe

In [54]:
# Save clean df to a csv
df.to_csv('Clean-ALL_reviews.csv', index=False)